## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-19-23-43-15 +0000,nyt,Trump Seeks to Oust U.S. Attorney Investigatin...,https://www.nytimes.com/2025/09/19/us/politics...
1,2025-09-19-23-43-00 +0000,wsj,Trump announced a move to dramatically reshape...,https://www.wsj.com/politics/policy/trump-to-a...
2,2025-09-19-23-40-27 +0000,wapo,Pentagon demands journalists pledge to not obt...,https://www.washingtonpost.com/business/2025/0...
3,2025-09-19-23-34-43 +0000,nyt,Ted Cruz Accuses FCC Chair of Mafia-Like Tacti...,https://www.nytimes.com/2025/09/19/us/politics...
4,2025-09-19-23-34-01 +0000,nyt,"As Farm Aid Reaches 40 Years, It Deals With Fa...",https://www.nytimes.com/2025/09/19/business/fa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2383/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,74
15,new,24
45,kimmel,15
141,he,13
389,up,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-09-19-23-04-00 +0000,wsj,President Trump said he wants to dismiss a fed...,https://www.wsj.com/politics/policy/trump-call...,171
152,2025-09-19-16-26-39 +0000,nypost,Florida judge strikes Trump’s $15B defamation ...,https://nypost.com/2025/09/19/us-news/florida-...,151
118,2025-09-19-18-27-14 +0000,cbc,Judge throws out Trump's $15B US lawsuit again...,https://www.cbc.ca/news/world/new-york-times-l...,145
113,2025-09-19-18-39-43 +0000,nyt,Judge Dismisses Trump’s $15 Billion Lawsuit Ag...,https://www.nytimes.com/2025/09/19/business/me...,142
104,2025-09-19-19-12-36 +0000,bbc,Judge strikes down 'superfluous' Trump lawsuit...,https://www.bbc.com/news/articles/c62n7025wdgo...,141


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,171,2025-09-19-23-04-00 +0000,wsj,President Trump said he wants to dismiss a fed...,https://www.wsj.com/politics/policy/trump-call...
48,75,2025-09-19-21-31-00 +0000,wsj,Senate Democrats blocked Republicans’ stopgap ...,https://www.wsj.com/politics/policy/government...
98,65,2025-09-19-19-37-40 +0000,nypost,"Ted Cruz rips FCC’s Brendan Carr, compares him...",https://nypost.com/2025/09/19/business/ted-cru...
75,46,2025-09-19-20-28-00 +0000,wsj,House lawmakers adopted a Republican-led resol...,https://www.wsj.com/politics/house-charlie-kir...
152,46,2025-09-19-16-26-39 +0000,nypost,Florida judge strikes Trump’s $15B defamation ...,https://nypost.com/2025/09/19/us-news/florida-...
78,41,2025-09-19-20-21-00 +0000,wsj,Trump administration officials are in discussi...,https://www.wsj.com/politics/national-security...
167,40,2025-09-19-15-45-33 +0000,bbc,Trump says Xi approves of TikTok deal as leade...,https://www.bbc.com/news/articles/c4g7l7yl832o...
81,39,2025-09-19-20-19-42 +0000,nypost,D4vd cancels world tour as cops investigate mu...,https://nypost.com/2025/09/19/us-news/d4vd-can...
6,38,2025-09-19-23-30-00 +0000,wsj,Health secretary Robert F. Kennedy Jr.’s handp...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
257,38,2025-09-19-09-30-00 +0000,wsj,The Fed’s rate cut will likely reduce U.S. bor...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
